In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

###### spark 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

###### Загрузка датасета 

In [3]:
data = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET, hive)

###### create story target items 

In [4]:
item_dict = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_ITEM_DICT, hive)

In [5]:
item_dict.filter("product <> '' and product <> 'none'").show(10)

,hitPagePath,eventCategory,eventAction,storyTarget,product,event_id,event_user_count,event_user_percent,group_id,group_user_count,group_user_percent,item_id,item_user_count,item_user_percent
0,/accounts-payments,[std]: asup: offers,click,None,"smart_credit, link-url: https://www.sberbank.ru/common/img/uploaded/files/pdf/s_m_business/cards/rules/rules_100_day_grace_from_sberbank.pdf, link-name: ссылке",33314281,3604,0.39574,None,None,None,33314281,3604,0.39574
1,/accounts-payments,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33314281,3604,0.39574,None,None,None,33314281,3604,0.39574
2,/accounts-payments/account/:accountId,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33318497,12065,1.32482,None,None,None,33318497,12065,1.32482
3,/accounts-payments/account/:accountId,[std]: asup: offers,click,None,"smart_credit, link-url: https://www.sberbank.ru/common/img/uploaded/files/pdf/s_m_business/cards/rules/rules_100_day_grace_from_sberbank.pdf, link-name: ссылке",33318497,12065,1.32482,None,None,None,33318497,12065,1.32482
4,/accounts-payments/orders/sign,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33332920,296,0.03250,None,None,None,33332920,296,0.03250
5,/accounts/kartoteka/:id/:cardType,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33333251,359,0.03942,None,None,None,33333251,359,0.03942
6,/acquiring/merchant/pos-list,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33333912,755,0.08290,None,None,None,33333912,755,0.08290
7,/acquiring/merchant/reports-list/report-designer-request/:id,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33335345,36,0.00395,None,None,None,33335345,36,0.00395
8,/acquiring/merchant/requests/pos-registration-request,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33335544,75,0.00824,None,None,None,33335544,75,0.00824
9,/cash-in-transit,[std]: asup: offers,click,None,"zarplata, link-url: https://www.sberbank.ru/ru/s_m_business/bankingservice/cards/salaryproject, link-name: подробнее на сайте",33335848,773,0.08488,None,None,None,33335848,773,0.08488


In [6]:
target_items = item_dict.filter("product <> '' and product <> 'none' and storyTarget <> 'none'")\
                            .select("item_id")\
                            .distinct()\
                            .toPandas()["item_id"]\
                            .tolist()

In [7]:
len(target_items)

242

In [8]:
item_dict.filter(f.col("item_id").isin(target_items)) \
            .groupBy("item_id").agg(f.first("product"), f.first("item_user_count")).show()

,item_id,"first(product, false)","first(item_user_count, false)"
0,125965657,korpkarta,68
1,125925370,merchant-acquiring,329
2,125930204,zarplata,4
3,125967271,customwarranty,1
4,125962114,e_inv_const_doc,2
5,125958154,e_inv,7
6,125968433,e-acquiring,111
7,125930028,smart_credit,41
8,125929450,smart_credit,9
9,125928030,zarplata,1


###### Разделение на train, val, test

In [9]:
data_target = data.filter(f.col("item_id").isin(target_items))
data_non_target = data.subtract(data_target)

print("Non target: {}\nTarget: {}".format(data_non_target.count(), 
                                          data_target.count()))

Non target: 2358607944
Target: 59309019


###### Test выборка 

In [10]:
test = data_target

In [11]:
train_target, test_target = data_target.randomSplit([0.8, 0.2])

In [12]:
print("Target:\n\ttrain:\t{}\n\ttest:\t{}".format(train_target.count(), 
                                                test_target.count()))

Target:
	train:	47451417
	test:	11857602


Объединяем все нецелевые события с обучающей частью целевых для обучающей выборки

In [13]:
train_union = data_non_target.union(train_target)

train_union.cache()
train_union.count()

2406059361

Так как мы хотим оценивать качество только по целевым событиям, то будем учитывать только целевые события в валидационной выборке

In [14]:
val = test_target

val.cache()
val.count()

11857602

###### Отношения количества активных целевых событий в train, val 

In [15]:
train_target_count = train_union.filter(f.col("item_id").isin(target_items)) \
                        .select(f.sum("rating").alias("count"))\
                        .toPandas()["count"][0]
        
val_target_count = val.filter(f.col("item_id").isin(target_items)) \
                        .select(f.sum("rating").alias("count"))\
                        .toPandas()["count"][0]

In [16]:
print("Target count:\n\tTrain: {:d}\n\tVal: {:d}\nRatio val/train: {:f}" \
            .format(int(train_target_count), int(val_target_count), 
                    val_target_count / train_target_count))

Target count:
	Train: 16301
	Val: 4017
Ratio val/train: 0.246427


###### Исключение событий из будущего в обучающей выборке 

Так как обучающая выборка содержит все нецелевые события, то она может содержать и события, которые произошли после целевых события из тестовой выборки. Таким образом по налищию этих события в обучающей выборке модель может узнать статус целевого события - т.е. произойдёт утечка данных.

Выберем активные целевые события из тестового датасета

In [17]:
val_pos = val.filter("rating = 1.0") \
                .withColumnRenamed("timestamp", "timestamp_test")\
                .select("user_id", "timestamp_test")
val_pos.count()

4017

Добавим ко всем событиям пользователей информацию о целевых событиях у того же пользователя

In [18]:
train_cross = train_union.join(val_pos, on="user_id", how="left_outer") \
                     .select("user_id", "item_id", "rating", "timestamp",
                             (f.col("timestamp") - f.col("timestamp_test")).alias("ts_diff"))
train_cross.count()

2406389502

Поставим флаг на события, которые могут являтся следствием целевых.

Критерии следственности:

Событие произошло после целевого
Разница между событиями меньше 10 минут

In [19]:
delete_range_min = 10
min_ts = -10**9
train_cons = train_cross.na.fill({"ts_diff": min_ts})\
                          .groupBy(train_union.columns).agg(f.min("ts_diff").alias("ts_diff"))\
                          .withColumn("is_consequence", (f.col("ts_diff") >= 0) &
                                                        (f.col("ts_diff") <= delete_range_min * 60)) \
                          .drop("ts_diff")
train_cons.show(5)

,user_id,item_id,timestamp,rating,is_consequence
0,471,57531037,1622449888,1.0,False
1,471,39016804,1622012412,1.0,False
2,471,588456,1622184883,1.0,False
3,471,35646684,1622449926,1.0,False
4,471,58242558,1622449890,1.0,False


Занулим все следственные события

In [20]:
train = train_cons.withColumn("rating_filt", f.when(~f.col("is_consequence"), f.col("rating")) \
                                                       .otherwise(0.0)) \
                            .drop("rating", "is_consequence") \
                            .withColumnRenamed("rating_filt", "rating")

In [21]:
train.show(5)

,user_id,item_id,timestamp,rating
0,471,57531037,1622449888,1.0
1,471,28715431,1622449927,1.0
2,471,39016804,1622012412,1.0
3,471,32083838,1622184886,1.0
4,471,46659237,1622012418,1.0


###### Статистика 

In [22]:
train_0 = train.filter("rating = 0.0").count()
train_1 = train.filter("rating = 1.0").count()

val_0 = val.filter("rating = 0.0").count()
val_1 = val.filter("rating = 1.0").count()

print("Train:\n\trating=1.0: {}\n\trating=0.0: {}".format(train_1, train_0))
print("Test:\n\trating=1.0: {}\n\trating=0.0: {}".format(val_1, val_0))

Train:
	rating=1.0: 7049306
	rating=0.0: 2399010055
Test:
	rating=1.0: 4017
	rating=0.0: 11853585


###### Коррекция тестовой выборки

In [23]:
train_pos = train.filter("rating = 1.0")

In [24]:
val_cor = val.join(train_pos.select("user_id").distinct() \
                              .withColumn("in_train", f.lit(True)),
                       on="user_id", 
                       how="left_outer") \
                .filter("in_train = True") \
                .drop("in_train")
        
test_cor = test.join(train_pos.select("user_id").distinct() \
                              .withColumn("in_train", f.lit(True)),
                       on="user_id", 
                       how="left_outer") \
                .filter("in_train = True") \
                .drop("in_train")

###### Сохранение данных 

In [25]:
train_table = RECSYS_STORY_TRAIN

drop_table(SBX_TEAM_DIGITCAMP, train_table, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, train_table, train, hive)

In [28]:
train_pos_table = RECSYS_STORY_DATASET_TRAIN_POS

drop_table(SBX_TEAM_DIGITCAMP, train_pos_table, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, train_pos_table, train_pos, hive)

In [29]:
train_val_table = "RECSYS_STORY_DATASET_VAL"

drop_table(SBX_TEAM_DIGITCAMP, train_val_table, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, train_val_table, val_cor, hive)

In [30]:
test_table = RECSYS_STORY_TEST

drop_table(SBX_TEAM_DIGITCAMP, test_table, hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, test_table, test_cor, hive)

In [34]:
load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_TRAIN, hive).show()

,user_id,item_id,timestamp,rating
0,471,28715431,1622449927,1.0
1,471,46659237,1622012418,1.0
2,471,1415429,1622012716,1.0
3,471,58242558,1622449890,1.0
4,471,41621837,1622012760,1.0
5,471,32083838,1622184886,1.0
6,471,319615,1622012704,1.0
7,471,588456,1622184883,1.0
8,471,35646684,1622449926,1.0
9,471,57531037,1622449888,1.0
